## IMPORTS

In [5]:
! conda activate myprojects
! pip install -r requirements.txt
! pip list

/bin/bash: line 1: conda: command not found
Package                  Version
------------------------ -----------
accelerate               1.3.0
aiohappyeyeballs         2.4.4
aiohttp                  3.11.11
aiosignal                1.3.2
annotated-types          0.7.0
anyio                    4.8.0
asttokens                3.0.0
attrs                    24.3.0
certifi                  2024.12.14
charset-normalizer       3.4.1
comm                     0.2.2
dataclasses-json         0.6.7
debugpy                  1.8.12
decorator                5.1.1
diskcache                5.6.3
executing                2.2.0
filelock                 3.17.0
frozenlist               1.5.0
fsspec                   2024.12.0
greenlet                 3.1.1
h11                      0.14.0
httpcore                 1.0.7
httpx                    0.28.1
httpx-sse                0.4.0
huggingface-hub          0.27.1
idna                     3.10
ipykernel                6.29.5
ipython                  8.31.0


In [ ]:
import main

## USAGE

TO RUN Benchmarking tests against a pre-defined set of questions and sets of pre-defined configurations on the LLM

In [7]:
! python main.py

/home/bot/personal_proj/moneylion-ml-assessment/main.py:57: LangChainDeprecationWarning: Importing SQLDatabase from langchain is deprecated. Please replace deprecated imports:

>> from langchain import SQLDatabase

with new imports of:

>> from langchain_community.utilities import SQLDatabase
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.sql_database import SQLDatabase
/home/bot/personal_proj/moneylion-ml-assessment/classes.py:7: LangChainDeprecationWarning: Importing SQLDatabase from langchain is deprecated. Please replace deprecated imports:

>> from langchain import SQLDatabase

with new imports of:

>> from langchain_community.utilities import SQLDatabase
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.sql_database import SQLDatabase
2025